In [1]:
from scipy import stats
from numpy.random import binomial
from numpy.random import normal

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind, ttest_rel
from scipy.stats import t, norm

In [3]:
df = pd.read_csv('data/data_ab.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2025-01-21 22:11:48.556739,control,old_page,0
1,804228,2025-01-12 08:01:45.159739,control,old_page,0
2,661590,2025-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2025-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2025-01-21 01:52:26.210827,control,old_page,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [6]:
print("Уникальные значения в 'group':", df['group'].unique())
print("Уникальные значения в 'landing_page':", df['landing_page'].unique())
print("Уникальные значения в 'converted':", df['converted'].unique())
df.duplicated('user_id').sum()

Уникальные значения в 'group': ['control' 'treatment']
Уникальные значения в 'landing_page': ['old_page' 'new_page']
Уникальные значения в 'converted': [0 1]


np.int64(3894)

# Проверка замысла A/B теста:
* ## Группа A (control + old_page):
  *  `group` = `control`

  *  `landing_page` = `old_page`

* ## Группа B (treatment + new_page):
  *  `group` = `treatment`

  *  `landing_page` = `new_page`

In [7]:

print("Всего строк:", len(df))
print(df['group'].value_counts())
print(df['landing_page'].value_counts())
cross = pd.crosstab(df['group'], df['landing_page'])
cross

Всего строк: 294478
group
treatment    147276
control      147202
Name: count, dtype: int64
landing_page
old_page    147239
new_page    147239
Name: count, dtype: int64


landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


In [ ]:
control_old = cross.loc['control', 'old_page']
control_new = cross.loc['control', 'new_page']
treatment_old = cross.loc['treatment', 'old_page']
treatment_new = cross.loc['treatment', 'new_page']

print(f"Control + Old: {control_old}")
print(f"Control + New: {control_new}")
print(f"Treatment + Old: {treatment_old}")
print(f"Treatment + New: {treatment_new}")

print(f"Ошибочных кимбинаций: {control_new + treatment_old}, {(control_new + treatment_old)/len(df)*100:.2f}%")

Control + Old: 145274
Control + New: 1928
Treatment + Old: 1965
Treatment + New: 145311
Ошибочных кимбинаций: 3893, 1.32%


Обнаружено 3893 пользователей (1.3%), у которых показанная версия страницы не совпадала с назначенной группой. Данные пользователя с ошибочной комбинацией (control+new_page или treatment+old_page) исключены из анализа, так как нарушают протокол эксперимента и могут искажать итоговый эффект.


In [9]:
df_clean = df[((df['group'] == 'control') & (df['landing_page'] == 'old_page')) | ((df['group'] == 'treatment') & (df['landing_page'] == 'new_page'))]
print("Пользователи несколько раз совершившие целевое действие:", df_clean.duplicated('user_id').sum())
df_clean = df_clean.drop_duplicates(subset='user_id', keep='first')

Пользователи несколько раз совершившие целевое действие: 1


При подготовке данных A/B-теста критически важно удалять дубликаты пользователей после фильтрации несоответствующих групп. Один пользователь, учтённый дважды, нарушает независимость наблюдений и искажает статистическую значимость результатов, что может привести к некорректным выводам по эффективности тестируемых изменений.